In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [9]:
train = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/train.csv')
test = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/test.csv')
trade = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/international_trade.csv')
sample = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/sample_submission.csv')

In [10]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

test['year'] = test['timestamp'].apply(lambda x : int(x[0:4]))
test['month'] = test['timestamp'].apply(lambda x : int(x[5:7]))
test['day'] = test['timestamp'].apply(lambda x : int(x[8:10]))

In [11]:
# train['timestamp'] = pd.to_datetime(train['timestamp'])
# trade['기간'] = pd.to_datetime(trade['기간'])

In [12]:
def english(data, col):
    data[col] = data[col].apply(lambda x: 'TG' if '감귤' in x else x)
    data[col] = data[col].apply(lambda x: 'BC' if '브로콜리' in x else x)
    data[col] = data[col].apply(lambda x: 'RD' if '무' in x else x)
    data[col] = data[col].apply(lambda x: 'CR' if '당근' in x else x)
    data[col] = data[col].apply(lambda x: 'CB' if '양배추' in x else x)
    
    return data
english(trade, '품목명')

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990
1,2019-01,양파,821330,222,4003206,1118,-896
2,2019-01,쪽파,60,1,93405,128,-127
3,2019-01,BC,160,1,638913,563,-562
4,2019-01,CB,0,0,7580,38,-38
...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71
1270,2023-02,사과,135165,351,0,0,351
1271,2023-02,배,2206012,5411,1,0,5411
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0


In [13]:
filtered = trade[(trade['품목명'] == 'TG') | (trade['품목명'] == 'BC') | (trade['품목명'] == 'RD') | (trade['품목명'] == 'CR') | (trade['품목명'] == 'CB')]
filtered

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
3,2019-01,BC,160,1,638913,563,-562
4,2019-01,CB,0,0,7580,38,-38
5,2019-01,CB,184650,94,395802,90,4
8,2019-01,CR,23150,22,7466150,2955,-2934
12,2019-01,RD,2627,23,94529,464,-441
...,...,...,...,...,...,...,...
1250,2023-02,CB,13188,13,377456,104,-91
1253,2023-02,CR,22510,20,9260020,3758,-3737
1254,2023-02,RD,4000,4,2,0,4
1258,2023-02,RD,1319,14,104566,454,-440


In [15]:
filtered['year'] = filtered['기간'].apply(lambda x : int(x[0:4]))
filtered['month'] = filtered['기간'].apply(lambda x : int(x[5:7]))
# filtered['day'] = filtered['기간'].apply(lambda x : int(x[8:10]))

In [16]:
filtered = filtered.rename(columns={'품목명':'item'})

In [17]:
suji = filtered.groupby(['month', 'item'])['무역수지'].mean()
suji = pd.DataFrame(suji)
ex_kg = filtered.groupby(['month', 'item'])['수출 중량'].mean()
ex_kg = pd.DataFrame(ex_kg)
ex_money = filtered.groupby(['month', 'item'])['수출 금액'].mean()
ex_money = pd.DataFrame(ex_money)
im_kg = filtered.groupby(['month', 'item'])['수입 중량'].mean()
im_kg = pd.DataFrame(im_kg)
im_money = filtered.groupby(['month', 'item'])['수입 금액'].mean()
im_money = pd.DataFrame(im_money)

In [18]:
combine = pd.concat([ex_kg, ex_money, im_kg, im_money, suji], axis=1).reset_index()
combine

,month,item,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,1,BC,4.640000e+01,0.400,6.827742e+05,751.800000,-751.200
1,1,CB,5.176180e+04,28.800,6.289534e+05,203.700000,-174.800
2,1,CR,2.289740e+04,29.400,6.910105e+06,2758.000000,-2729.000
3,1,RD,1.027600e+03,10.400,7.145580e+04,318.000000,-307.600
4,1,TG,1.926216e+05,317.400,2.000000e-01,0.000000,317.400
5,2,BC,4.232000e+02,1.000,4.037452e+05,432.800000,-431.800
6,2,CB,3.084360e+04,15.300,3.319038e+05,127.000000,-111.700
7,2,CR,1.752400e+04,17.200,6.150161e+06,2829.000000,-2812.200
8,2,RD,1.559167e+03,9.500,5.313050e+04,238.166667,-228.500
9,2,TG,2.370640e+04,112.000,1.548120e+04,51.400000,60.600


In [96]:
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5
...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2


In [112]:
train2 = pd.merge(train, combine, on=['month', 'item'])
train2

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,192621.6,317.40,0.2,0.0,317.4
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,192621.6,317.40,0.2,0.0,317.4
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,192621.6,317.40,0.2,0.0,317.4
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,192621.6,317.40,0.2,0.0,317.4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,192621.6,317.40,0.2,0.0,317.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,BC_E_S_20221227,2022-12-27,BC,E,S,3144.0,2138.0,2022,12,27,879.0,2.25,737942.5,891.5,-889.5
59393,BC_E_S_20221228,2022-12-28,BC,E,S,1552.0,2137.0,2022,12,28,879.0,2.25,737942.5,891.5,-889.5
59394,BC_E_S_20221229,2022-12-29,BC,E,S,896.0,2608.0,2022,12,29,879.0,2.25,737942.5,891.5,-889.5
59395,BC_E_S_20221230,2022-12-30,BC,E,S,528.0,3006.0,2022,12,30,879.0,2.25,737942.5,891.5,-889.5


In [114]:
test2 = pd.merge(test, combine, on=['month', 'item'])
test2

,ID,timestamp,item,corporation,location,year,month,day,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,7207.5,33.75,4665.5,14.5,19.25
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,7207.5,33.75,4665.5,14.5,19.25
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,7207.5,33.75,4665.5,14.5,19.25
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,7207.5,33.75,4665.5,14.5,19.25
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,7207.5,33.75,4665.5,14.5,19.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,BC_E_S_20230327,2023-03-27,BC,E,S,2023,3,27,45.5,1.00,753128.5,819.5,-818.50
1088,BC_E_S_20230328,2023-03-28,BC,E,S,2023,3,28,45.5,1.00,753128.5,819.5,-818.50
1089,BC_E_S_20230329,2023-03-29,BC,E,S,2023,3,29,45.5,1.00,753128.5,819.5,-818.50
1090,BC_E_S_20230330,2023-03-30,BC,E,S,2023,3,30,45.5,1.00,753128.5,819.5,-818.50


In [121]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

y = train['price(원/kg)']
X = train2.drop(['ID', 'timestamp', 'supply(kg)', 'price(원/kg)'], axis=1)
test_x = test2.drop(['ID', 'timestamp'], axis=1)
test_x = pd.get_dummies(test_x)
X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)
test_x_sc = scaler.transform(test_x)

# 랜덤포레스트 회귀 모델 생성
rf = RandomForestRegressor(n_estimators=500, random_state=42, max_depth=7)

# 모델 학습
rf.fit(X_train_sc, y_train)

# 예측
y_pred = rf.predict(X_test_sc)

# 모델 평가
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print('mape:', mean_absolute_percentage_error(y_test, y_pred))

Mean Squared Error: 2158534.7613108475
mape: 1.6526897749482665e+18


In [123]:
mean_absolute_error(y_test, y_pred)
mean_absolute_percentage_error(y_test, y_pred)

1.6526897749482665e+18

In [129]:
from sklearn.model_selection import GridSearchCV

# 그리드 서치할 매개변수를 설정합니다.
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 랜덤포레스트 회귀 모델을 생성합니다.
rf = RandomForestRegressor()

# 그리드 서치 객체를 생성합니다.
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)

# 그리드 서치를 수행합니다.
grid_search.fit(X_train_sc, y_train)

# 최적의 매개변수와 점수를 출력합니다.
print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters found:  {'max_depth': 9, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 500}
Best score:  0.559238161669945


In [132]:
rfr = RandomForestRegressor(max_depth=9, n_estimators=500, min_samples_split=4)


rfr.fit(X_train_sc, y_train)
test_pred = rfr.predict(test_x_sc)
test_pred = pd.DataFrame(test_pred)
sample = pd.concat([sample, test_pred], axis=1).drop('answer', axis=1)
sample = sample.rename(columns={0: 'answer'})

test_pred.to_csv('submission_jeju_3.csv', index=False)

In [127]:
pred = rf.predict(test_x_sc)
pred = pd.DataFrame(pred)
submission = pd.concat([sample, pred], axis=1)
submission.drop('answer', axis=1, inplace=True)
submission = submission.rename(columns={0: 'answer'})

submission.to_csv('submission_km_univ.csv', index=False)

In [29]:
# 필요한 라이브러리를 불러옵니다.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 데이터 전처리
# (필요한 전처리 과정을 수행합니다.)


# 특성 선택
X = filtered[['품목명', '수출 중량', '수출 금액', '수입 중량', '수입 금액']]

# 라벨 선택
y = filtered['무역수지']

X = pd.get_dummies(X)

# 데이터를 훈련 세트와 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 회귀 모델 선택 및 학습
model = RandomForestRegressor()
model.fit(X_train, y_train)

# 모델 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 새 데이터에 대한 원핫인코딩
new_data = pd.DataFrame({'품목명_CB': [0], '품목명_CR': [0], '품목명_RD': [0], '품목명_TG': [1], '수출 중량': [1000], '수출 금액': [50000], '수입 중량': [800], '수입 금액': [40000]})

# 예측할 데이터의 형태를 훈련 데이터와 동일하게 맞춤
new_data = new_data.reindex(columns=X.columns, fill_value=0)

# 새 데이터에 대한 예측
predicted_result = model.predict(new_data)

# 예측 결과를 출력합니다.
print(f"3월 무역 수지 예측 결과: {predicted_result}")



Mean Squared Error: 5107.035283870963
3월 무역 수지 예측 결과: [-4412.06]


In [38]:
filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 306 entries, 3 to 1264
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   기간      306 non-null    datetime64[ns]
 1   품목명     306 non-null    object        
 2   수출 중량   306 non-null    int64         
 3   수출 금액   306 non-null    int64         
 4   수입 중량   306 non-null    int64         
 5   수입 금액   306 non-null    int64         
 6   무역수지    306 non-null    int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 19.1+ KB


In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# 머신 러닝 모델 학습을 위한 데이터 전처리
X = trade[['수출 중량', '수출 금액', '수입 중량', '수입 금액']]  # 독립 변수
y = trade['무역수지']  # 종속 변수

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 선형 회귀 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 2023-03의 품목명 데이터를 가져와서 예측하기
new_df = test
# test 데이터프레임에 열 추가하기
test['수출 중량'] = 0
test['수출 금액'] = 0
test['수입 중량'] = 0
test['수입 금액'] = 0

# 예측을 위한 데이터프레임 준비
new_df = test[['수출 중량', '수출 금액', '수입 중량', '수입 금액']]


predicted_results = model.predict(new_df[['수출 중량', '수출 금액', '수입 중량', '수입 금액']])

predicted_results


array([0.01012854, 0.01012854, 0.01012854, ..., 0.01012854, 0.01012854,
       0.01012854])

In [60]:
predicted_results = pd.DataFrame(predicted_results)
test = pd.concat([test, predicted_results], axis=1)
test.drop(['수출 중량', '수출 금액', '수입 금액', '수입 중량'], axis=1, inplace=True)
test

,ID,timestamp,item,corporation,location,year,month,day,0
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,0.010129
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,0.010129
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,0.010129
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,0.010129
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,0.010129
...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,2023,3,27,0.010129
1088,RD_F_J_20230328,2023-03-28,RD,F,J,2023,3,28,0.010129
1089,RD_F_J_20230329,2023-03-29,RD,F,J,2023,3,29,0.010129
1090,RD_F_J_20230330,2023-03-30,RD,F,J,2023,3,30,0.010129


In [62]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# 머신 러닝 모델 학습을 위한 데이터 전처리
X = trade[['수출 중량', '수출 금액', '수입 중량', '수입 금액']]  # 독립 변수
y = trade['무역수지']  # 종속 변수

# 선형 회귀 모델 학습
model = LinearRegression()
model.fit(X, y)

# 2023-03의 품목명 데이터를 가져와서 예측하기

new_df = test
predicted_results = model.predict(new_df[['수출 중량', '수출 금액', '수입 중량', '수입 금액']])

print(predicted_results)


[0.00833921 0.00833921 0.00833921 ... 0.00833921 0.00833921 0.00833921]
